# RAMOO 2025
## TU Munich @ Straubing

\begin{aligned}
\text{maximize} \quad & f(x) = \big[ f_1(x),...,f_d(x) \big] \\
\text{subject to} \quad & x \in X
\end{aligned}

----

## An instance of bi-objective 01 unidimensional knapsack 

In [ ]:

using Random

function generate_MO01UKP(n = 10, d = 2, max_ci = 100, max_wi = 30)
    Random.seed!(123)
    
    p = rand(1:max_ci,d,n)      # c_i \in [1,max_ci]   # profits
    w = rand(1:max_wi,n)        # w_i \in [1,max_wi]   # weight
    c = round(Int64, sum(w)/2)                         # capacity
                
    return p, w, c
end


In [ ]:

p,w,c = generate_MO01UKP(10,2)
d,n = size(p) 


-----

## Basics with `JuMP` and `MultiObjectiveAlgorithms`

#### Lex-optimal with `MultiObjectiveAlgorithms`

In [ ]:

using JuMP, GLPK
import MultiObjectiveAlgorithms as MOA

model = Model()
@variable(model, x[1:n], Bin)
@expression(model, z[k=1:d], sum(p[k,j] * x[j] for j in 1:n))
@objective(model, Max, [z[k] for k=1:d])
@constraint(model, sum(w[i] * x[i] for i in 1:n) ≤ c)

set_optimizer(model,()->MOA.Optimizer(GLPK.Optimizer))
set_attribute(model, MOA.Algorithm(), MOA.Lexicographic())
optimize!(model)

z1_opt = objective_value(model; result=2)
z2_opt = objective_value(model; result=1)
z_ideal = objective_bound(model)

println("zᴵ     = ",z_ideal,"\n")
println("z¹_lex = ", z1_opt, "\nz²_lex = ", z2_opt)  


#### $\epsilon$-constraint with `MultiObjectiveAlgorithms`

In [ ]:

set_attribute(model, MOA.Algorithm(), MOA.EpsilonConstraint())
optimize!(model)

for i in 1:result_count(model)
    z_opt = objective_value(model; result=i)
    println("z_nd = ", z_opt)
end


-----

# Augmented Weighted Tchebycheff

    Compute one nondominated point by application of the Tchebycheff theory.
    Application to the bi-objective 01 unidimensionnal knapsack problem. 
        
    In  Steuer, R.E., Choo, EU. An interactive weighted Tchebycheff procedure 
        for multiple objective programming. Mathematical Programming 26, 326–344 (1983).   
        


$\ $ The scalarizing function $s(X, \lambda, r^{*}, \rho)$:
    
\begin{aligned}
\text{minimize} \quad & \bigg[ \max_{k=1,\ldots,d} \big\{ \lambda_k \big( r_k^{*} - f_k(x) \big\} 
+ \rho \sum_{k=1}^d \big( r_k^{*} - f_k(x) \big) \bigg]\\
\text{subject to} \quad & x \in X
\end{aligned}

#### Modeling, solving, querying and formatting the results

In [ ]:

using JuMP, GLPK


In [ ]:

function  SolveAugmentedWeightedTchebycheff(  
            solver::DataType, 
            p::Matrix{Int64},  w::Vector{Int64},  c::Int64,  
            λ::Vector{Float64}, rp::Vector{Int64}, ρ::Float64 )
           
    d,n = size(p)
    model = Model(solver)  

    # settings related to the bi-01UKP model    
    @variable(model, x[1:n], Bin)                              
    @constraint(model, sum(w[i] * x[i] for i in 1:n) ≤ c)        
    @expression(model, z[k=1:d], sum(p[k,j] * x[j] for j in 1:n))   

    # settings related to the Tchebycheff model    
    @variable(model, α ≥ 0)        
    @constraint(model, con[k=1:d], α ≥  λ[k] * (rp[k] -  z[k]))
    @objective(model, Min, α + ρ * sum((rp[k] -  z[k]) for k=1:d))
        
    # optimize the scalarizing function
    set_silent(model)
    solve_time_sec = 0.0
    optimize!(model)
    @assert is_solved_and_feasible(model) "Error: optimal solution not found"
    solve_time_sec += round( solve_time(model) , digits=3)

    # querying and formatting the results    
    f_Opt = [ round(Int, sum(p[k,j] * value(x[j]) for j in 1:n) ) for k in 1:d ]
    x_Opt = [ round(Int, value(x[i])) for i in 1:n ]       

    return f_Opt, x_Opt , solve_time_sec
end


#### Example of computing one point $y \in Y_N$ with $s(X, \lambda, r^{*}, \rho)$

In [ ]:

p,w,c = generate_MO01UKP()

λ  = [0.25,0.75]
rp = Int.(z_ideal) .+ 1
ρ  = 0.001

solver = GLPK.Optimizer

z,x,t = SolveAugmentedWeightedTchebycheff(solver, p,w,c, λ,rp,ρ)

println("zOpt = ", z)
println("XOpt = ", x)
println("time = $t (s)") 


#### Example of a naive and not certified approach for computing the set $Y_N$ with $s(X, \lambda, r^{*}, \rho)$

In [ ]:

using Printf

zPrec = [NaN,NaN]
for step in 0.01:0.01:0.99
    λ =[step,1.0-step] 
    z,x,t = SolveAugmentedWeightedTchebycheff(solver, p,w,c, λ,rp,ρ)
    if z != zPrec 
        zPrec = z
        @printf("  λ=(%6.3f %6.3f)   zND= (%3d %3d) \n",λ[1],λ[2], z[1],z[2])
    end
end


-----

## Algorithm of *Dachert-Gorski-Klamroth* (2012) for computing the set $Y_N$ 

    In  Kerstin Dächert, Jochen Gorski, Kathrin Klamroth. An augmented weighted 
        Tchebycheff method with adaptively chosen parameters for discrete 
        bicriteria optimization problems. Computers & Operations Research,
        Volume 39, Issue 12, Pages 2929-2943, 2012.
        

<img src="img/AlgoKerstin.png" alt="Description" width="600" style="margin-left:60px;" />

In [ ]:
"""
    Sequential algorithm for generating the nondominated set based on 
    an adaptive_augmented_tchebycheff. 
    Application to the bi-objective 01 unidimensionnal knapsack problem. 
"""
function DachertGorskiKlamroth_knapsack(  
            solver::DataType, 
            p::Matrix{Int64},  w::Vector{Int64},  c::Int64
        )

    # Create two empty sorted lists ND and Temp
    ND   = (Vector{Int64})[]
    Temp = (Vector{Int64})[]

    # Compute lexicographic optimal solutions z1 and z2 wrt. f1 and f2, respectively, and zᴵ, the ideal point
    z1, z2, zᴵ = lex_optimize_knapsack(solver, p,w,c)
    println( "  z1: ", z1, "   z2: ", z2, "   zᴵ: ", zᴵ, "\n")

    if z1 == z2
        # the set of nondominated points is composed of a single point 
        push!(ND, z1)
        return ND
    else
        # Let's play...
        push!(Temp, z1); push!(Temp, z2); #push!(ND, z1)

        while length(Temp) ≥ 2
            # Compute parameters and optionally the new reference point wrt. the first two solutions z1,z2 in Temp
            z1 = Temp[1]; z2 = Temp[2]

            # Solve the resulting subproblem and find solution z
            z  = solve_subproblem_knapsack(solver, p,w,c, z1,z2)

            if z == z1 || z == z2
                # Remove first element from Temp and insert it as last element to ND
                popfirst!(Temp);  push!(ND, z1)
            else
                # Insert z as new second element between z1 and z2 to Temp
                insert!(Temp, 2, z)
            end
        end

        # Add final element of Temp as last element to ND
        push!(ND, Temp[1])

        return ND
    end
end

In [ ]:
"""
    Compute the lexicographic optimal solution for the bi-objective 01UKP
"""
function lex_optimize_knapsack( 
            solver::DataType,             
            p::Matrix{Int64},  w::Vector{Int64},  c::Int64 )
    
    d,n = size(p)
    model = Model()
    @variable(model, x[1:n], Bin)
    @expression(model, z[k=1:d], sum(p[k,j] * x[j] for j in 1:n))
    @objective(model, Max, [z[k] for k=1:d])
    @constraint(model, sum(w[i] * x[i] for i in 1:n) ≤ c)

    set_optimizer(model,()->MOA.Optimizer(solver))
    set_attribute(model, MOA.Algorithm(), MOA.Lexicographic())
    optimize!(model)

    z1_opt = round.(Int, objective_value(model; result=2))
    z2_opt = round.(Int, objective_value(model; result=1))
    z_ideal = round.(Int, objective_bound(model))
    
    return z1_opt, z2_opt, z_ideal
end


In [ ]:
"""
    Solve the Augmented Weighted Tchebycheff subproblem
    using the Adaptative method.
"""
function solve_subproblem_knapsack(  
            solver::DataType, 
            p::Matrix{Int64},  w::Vector{Int64},  c::Int64,  
            z1::Vector{Int64}, z2::Vector{Int64} )

    rp  = zeros(Int,2);    # reference point
    λ   = zeros(2);        # weights

    x = z2[1]; y = z1[2]   # ideal point
    x = x+1;   y = y+1     # utopian point (dominates the ideal point)

    rp[1] = x; rp[2] = y   # setting the reference point with the local utopian point

    print("  Box:  z1=", z1, "  z2=", z2, "  ⇒  RP=($x, $y)   ⟶   ")

    # Compute adaptive parameters
    λ[1], λ[2], ρ = compute_adaptive_parameters(x, y)   

    # Solve the augmented weighted Tchebycheff problem
    z,x,t = SolveAugmentedWeightedTchebycheff(solver, p,w,c, λ,rp,ρ)

    @printf("λ₁=%6.4f  λ₂=%6.4f   ρ= %8.6f   ⟶   z=(%4d,%4d) \n",  λ[1], λ[2], ρ, z[1], z[2])

    return z
end


In [ ]:
"""
    Compute the adaptive parameters as described in Table 1 of
    the paper Dächert et al., 2012
"""
function compute_adaptive_parameters( 
            x::Int64, y::Int64 ) 
    
    if x > y && y ≥ 2
        denom = x*y - y - 3*x + x^2
        w1 = ( x*y - x - y ) / denom
        w2 = ( x*(x - 2)   ) / denom
        ρ  = x / denom
        
    elseif y > x && x ≥ 2
        denom = x*y - x - 3*y + y^2
        w1 = ( y * (y - 2) )  / denom
        w2 = (x*y - x - y) / denom
        ρ  = y / denom

    elseif x == y && y > 2
        w1 = 0.5
        w2 = 0.5
        ρ  = 1.0 / (2*(x - 2))
    else
        error("Unexpected case")
    end

    return w1, w2, ρ
end


In [ ]:

solver = GLPK.Optimizer
p,w,c = generate_MO01UKP()

YN = DachertGorskiKlamroth_knapsack(solver, p,w,c)

println("\n  Set of nondominated points found:")
println("    ", YN)
